# Aspect Based Sentiment Analysis (ABSA)

Code is stolen from: <br>
    https://medium.com/analytics-vidhya/aspect-based-sentiment-analysis-a-practical-approach-8f51029bbc4a <br>
    https://towardsdatascience.com/aspect-based-sentiment-analysis-using-spacy-textblob-4c8de3e0d2b9

### Standard setup stuff

In [24]:
# import the v importante libraries #likeapro
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer 
import stanza
import unidecode
from autocorrect import Speller


In [25]:
# download stuff
stanza.download('en') # download English model
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

2022-02-10 11:32:08 INFO: Downloading default packages for language: en (English)...
2022-02-10 11:32:09 INFO: File exists: C:\Users\natas\stanza_resources\en\default.zip.
2022-02-10 11:32:14 INFO: Finished downloading models and saved to C:\Users\natas\stanza_resources.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\natas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\natas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\natas\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\natas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Load csv file with customer data (amazon)

In [26]:
df = pd.read_csv('datafiniti-consumer-reviews-of-amazon-products/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv')

In [27]:
df.head(2)

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,NaN,0,3,http://reviews.bestbuy.com/3545/5442403/review...,I thought it would be as big as small paper bu...,Too small,llyyue,https://www.newegg.com/Product/Product.aspx%25...
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,allnewkindleereaderblack6glarefreetouchscreend...,...,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/review...,This kindle is light and easy to use especiall...,Great light reader. Easy to use at the beach,Charmi,https://www.newegg.com/Product/Product.aspx%25...


In [28]:
#clean up data to only have the important and fun stuff
df = df[['name', 'brand',
       'categories', 'primaryCategories', 'imageURLs', 'manufacturer',
       'manufacturerNumber', 'reviews.date',
       'reviews.dateSeen', 'reviews.doRecommend',
       'reviews.numHelpful', 'reviews.rating',
       'reviews.text','sourceURLs']]

df.head(2)

,name,brand,categories,primaryCategories,imageURLs,manufacturer,manufacturerNumber,reviews.date,reviews.dateSeen,reviews.doRecommend,reviews.numHelpful,reviews.rating,reviews.text,sourceURLs
0,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,Amazon,B00ZV9PXP2,2017-09-03T00:00:00.000Z,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,0,3,I thought it would be as big as small paper bu...,https://www.newegg.com/Product/Product.aspx%25...
1,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,Amazon,B00ZV9PXP2,2017-06-06T00:00:00.000Z,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,0,5,This kindle is light and easy to use especiall...,https://www.newegg.com/Product/Product.aspx%25...


In [29]:
df['custom_review_id'] = list(range(len(df)))

In [30]:
# check out this specific review - nnice
df['reviews.text'][3]

"I am 100 happy with my purchase. I caught it on sale at a really good price. I am normally a real book person, but I have a 1 year old who loves ripping up pages. The Kindle prevents that, it's extremely portable (it fits better in my purse than a giant book), and I have it loaded with lots of books. I finish one and start another, without having to go store. It serves all my needs. I picked this one over the Paperwhite because the price was unbeatable and the only difference that I could see was this one wasn't backlit. A simple book light from the Dollar tree solves that issue. This is my second Kindle (the first being the old Keyboard model, which I put down because I fell out of love with the keyboard. Lol) and it most likely won't be my last."

## Load csv file with customer data (restaurant reviews)

## Cleaning up in gross text

##### Basic preprocessing
- Remove punctuation marks
- Remove numeric data
- (Convert reviews to lowercase) #doesn't seem to matter

##### Advanced preprocessing
- Stopword Removal
- (Correction of spelling) #sometimes wrong -> changes meaning
- Stemming
- (Lemmatization) -> nope not a good idea
- Non English review removal

#### Text Cleaning functions <br>

In [31]:
from text_cleaning_functions import clean_text_completely, remove_whitespace

In [32]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

df['clean_review'] = df['reviews.text'].apply(lambda x: clean_text_completely(x, w_tokenizer, lemmatizer))

In [33]:
df.head()

,name,brand,categories,primaryCategories,imageURLs,manufacturer,manufacturerNumber,reviews.date,reviews.dateSeen,reviews.doRecommend,reviews.numHelpful,reviews.rating,reviews.text,sourceURLs,custom_review_id,clean_review
0,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,Amazon,B00ZV9PXP2,2017-09-03T00:00:00.000Z,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,0,3,I thought it would be as big as small paper bu...,https://www.newegg.com/Product/Product.aspx%25...,0,[I thought it would be as big as small paper b...
1,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,Amazon,B00ZV9PXP2,2017-06-06T00:00:00.000Z,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,0,5,This kindle is light and easy to use especiall...,https://www.newegg.com/Product/Product.aspx%25...,1,[This kindle is light and easy to use especial...
2,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,Amazon,B00ZV9PXP2,2018-04-20T00:00:00.000Z,2018-05-27T00:00:00Z,True,0,4,Didnt know how much i'd use a kindle so went f...,https://www.newegg.com/Product/Product.aspx%25...,2,[Didnt know how much i would use a kindle so w...
3,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,Amazon,B00ZV9PXP2,2017-11-02T17:33:31.000Z,2018-10-09T00:00:00Z,True,3,5,I am 100 happy with my purchase. I caught it o...,https://www.newegg.com/Product/Product.aspx%25...,3,"[I am 100 happy with my purchase, I caught it..."
4,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,Amazon,B00ZV9PXP2,2018-04-24T00:00:00.000Z,2018-05-27T00:00:00Z,True,0,5,Solid entry level Kindle. Great for kids. Gift...,https://www.newegg.com/Product/Product.aspx%25...,4,"[Solid entry level Kindle, Great for kids, G..."


## Linguistics intuition

In order to make some proper professional NLP we need to do some POS tagging (Part-of-Speech tagging) <br>
See spaCys documentation: https://spacy.io/usage/linguistic-features#pos-tagging

In [34]:
# We get started by importing spacy
import spacy
nlp = spacy.load("en_core_web_sm")

In [35]:
sentence_to_plot = df['clean_review'][10][0]
doc_to_plot = nlp(sentence_to_plot)
spacy.displacy.render(doc_to_plot, style="dep")

In [36]:
# explain specific tags:
print(spacy.explain("PRON"))
print(spacy.explain("VERB"))
print(spacy.explain("ADV"))

print("\n")

print(spacy.explain("nsubj"))
print(spacy.explain("acomp"))
print(spacy.explain("npadvmod"))

pronoun
verb
adverb


nominal subject
adjectival complement
noun phrase as adverbial modifier


# Time for the fun stuff

## Aspect detection

In [37]:
# we want to figure out the aspects in each sentence 

def detect_aspects(sentences):
    aspects = []
    
    for sentence in sentences:
        doc = nlp(sentence)
        descriptive_term = '' # ('good', bad, excellent etc.)
        target = '' #the target of the sentence ('Kindle, device etc.')
        for token in doc:
            #print(f"token: {token}, token.dep_:{token.dep_}, token.pos_:{token.pos_}")
            # if token is a noun 
            if (token.dep_ in ['nsubj']) and token.pos_ in ['NOUN']:
                target = token.text

            if token.pos_ == 'ADJ':
                prepend = ''
                for child in token.children:
                    if child.pos_ not in ['ADV']:
                        continue
                    prepend += child.text + ' '
                descriptive_term = prepend + token.text

        aspects.append({'aspect': target,
                    'description': descriptive_term})
    return(aspects)

#### Test the aspect and sentiment detection

In [38]:
test = [
  'This chocolate truffle cake is really tasty',
  'This party is amazing!',
  'My mom is the best!',
  'App response is very slow!',
  'The trip to India was not very enjoyable'
]

In [39]:
detect_aspects(test)

[{'aspect': 'cake', 'description': 'really tasty'},
 {'aspect': 'party', 'description': 'amazing'},
 {'aspect': 'mom', 'description': 'best'},
 {'aspect': 'response', 'description': 'very slow'},
 {'aspect': 'trip', 'description': 'very enjoyable'}]

### Apply aspect/sentiment detection to real data

In [40]:
df['aspect_sentiments'] = df['clean_review'].apply(detect_aspects)

In [41]:
check_comment_num = 10
print(df.iloc[check_comment_num]['reviews.text'])
df.iloc[check_comment_num]['aspect_sentiments']

If you really want to have a good read on something digital this is the item you're looking for. It really is lightweight and highly functional. The battery lasts a long time and the screen is easy on the eyes. The only difficulty sometimes is reading in extremely bright environments. Overall an excellent buy.


[{'aspect': '', 'description': 'digital'},
 {'aspect': '', 'description': 'highly functional'},
 {'aspect': 'screen', 'description': 'easy'},
 {'aspect': 'difficulty', 'description': 'extremely bright'},
 {'aspect': '', 'description': 'excellent'},
 {'aspect': '', 'description': ''}]

## Sentiment Analysis (sentiment score)

In [42]:
from textblob import TextBlob

In [43]:
# very simple mapping
def sentiment_score_to_category(polarity):
    if polarity < 0:
        return 'Negative'
    elif polarity == 0:
        return 'Neutral'
    else:
        return 'Positive'

In [44]:
def get_sentiment(aspect_list):
    for aspect in aspect_list:
        aspect['polarity'], aspect['subjectivity'] = TextBlob(aspect['description']).sentiment
        aspect['classification'] = sentiment_score_to_category(aspect['polarity'])
    return aspect_list

In [45]:
# get the aspect/sentiment for each sentence in each comment
df['aspect_sentiments'] = df['aspect_sentiments'].apply(get_sentiment)

In [46]:
df.head(2)

,name,brand,categories,primaryCategories,imageURLs,manufacturer,manufacturerNumber,reviews.date,reviews.dateSeen,reviews.doRecommend,reviews.numHelpful,reviews.rating,reviews.text,sourceURLs,custom_review_id,clean_review,aspect_sentiments
0,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,Amazon,B00ZV9PXP2,2017-09-03T00:00:00.000Z,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017...",False,0,3,I thought it would be as big as small paper bu...,https://www.newegg.com/Product/Product.aspx%25...,0,[I thought it would be as big as small paper b...,"[{'aspect': '', 'description': 'small', 'polar..."
1,"Amazon Kindle E-Reader 6"" Wifi (8th Generation...",Amazon,"Computers,Electronics Features,Tablets,Electro...",Electronics,https://pisces.bbystatic.com/image2/BestBuy_US...,Amazon,B00ZV9PXP2,2017-06-06T00:00:00.000Z,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017...",True,0,5,This kindle is light and easy to use especiall...,https://www.newegg.com/Product/Product.aspx%25...,1,[This kindle is light and easy to use especial...,"[{'aspect': 'kindle', 'description': 'easy', '..."


## Make Aspect dataframe

In [47]:
aspect_df = pd.DataFrame(df[['custom_review_id','aspect_sentiments']].explode().apply(pd.Series))
aspect_df.head()

TypeError: explode() missing 1 required positional argument: 'column'

### Aspects tied to comments dataframe

In [ ]:
df1 = pd.DataFrame(df.explode(column='aspect_sentiments'))
df1 = pd.concat([df1, df1['aspect_sentiments'].apply(pd.Series)], axis=1)

df_by_reviews = df1.groupby(['custom_review_id'], as_index = False).agg({'reviews.text':'min','aspect': ' '.join, 'description': ' '.join, 'polarity':np.mean, 'subjectivity':np.mean})
df_by_reviews.head(3)

## WordCloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

### All words

In [ ]:
allWords = ' '.join(review for review in aspect_df['aspect'])
allWords = remove_whitespace(allWords)

wordCloud = WordCloud(random_state=21, max_font_size=119).generate(allWords)
plt.imshow(wordCloud, interpolation='bilinear')
plt.axis('off')
plt.show()

NameError: name 'aspect_df' is not defined

### Negative Words

### Positive Words